This is code to enable us to connect to Postgres from Python. 

First, need to have the proper library installed. 

In [5]:
%pip install psycopg2
%pip install pandas

Note: you may need to restart the kernel to use updated packages.
  Using cached numpy-2.3.3-cp313-cp313-macosx_14_0_x86_64.whl.metadata (62 kB)
  Using cached pytz-2025.2-py2.py3-none-any.whl.metadata (22 kB)
  Using cached tzdata-2025.2-py2.py3-none-any.whl.metadata (1.4 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.5/11.5 MB 8.7 MB/s  0:00:01 eta 0:00:01
Using cached numpy-2.3.3-cp313-cp313-macosx_14_0_x86_64.whl (6.6 MB)
Using cached pytz-2025.2-py2.py3-none-any.whl (509 kB)
Using cached tzdata-2025.2-py2.py3-none-any.whl (347 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4/4 [pandas]2m3/4 [pandas]
Note: you may need to restart the kernel to use updated packages.


Add import statements.

In [1]:
import psycopg2
from psycopg2 import sql

Add database connection information. (Change according to your local situation, specifically, change the password.)

In [2]:
db_config = {
        "dbname": "prods",           # CHANGE if you are using a different database
        "user": "postgres",                
        "password": "password",   # CHANGE to your password
        "host": "localhost",            # or remote host IP address
        "port": 5432                    # default Postgres port
    }

Build query.

In [3]:
query = "SELECT * FROM rep"

Connect and run query. (Note exception handling.)

In [4]:
try:
    # Connect to PostgreSQL
    conn = psycopg2.connect(**db_config)
    cur = conn.cursor()

    # Run the SQL query
    cur.execute(query)

    # If it's a SELECT, fetch results
    if query.strip().lower().startswith("select"):
        result = cur.fetchall()
        for row in result:
            print(row)
    else:
        # Commit for INSERT/UPDATE/DELETE
        conn.commit()
        print("Query executed successfully.")

    # Close cursor and connection
    cur.close()
    conn.close()

except Exception as e:
    print("Error:", e)

('223', 'Roma', 'Theresa', '735 First', 'Leonard', 'GA', '23342', Decimal('21756.50'), Decimal('0.06'))
('237', 'Miller', 'Fred', '643 High', 'Sheldon', 'GA', '23553', Decimal('38612.00'), Decimal('0.08'))
('268', 'Rodriguez', 'Miguel', '2737 Tyler', 'Springfield', 'GA', '23342', Decimal('19774.00'), Decimal('0.06'))


Better formatted data using Pandas DF.

In [5]:
import pandas as pd

def read_sql_df(query: str) -> pd.DataFrame:
    try:
    # Connect to PostgreSQL
        conn = psycopg2.connect(**db_config)
        return pd.read_sql_query(query, conn)
        
    except Exception as e:
        print("Error:", e)

df = read_sql_df("SELECT * FROM customer")
df

/var/folders/2t/tt8wnk4n2s1_hpslcj1y5jj40000gn/T/ipykernel_50953/990575015.py:7: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  return pd.read_sql_query(query, conn)


,cust_id,cust_name,cust_street,cust_city,cust_state,cust_zip,cust_balance,cust_limit,rep_id
0,1159,Charles Appliance and Sport,3948 Brown,Leonard,GA,23342,5560.00,7500.0,223
1,1193,Streaming Direct,4938 Maple,Grove,GA,23321,210.40,10000.0,237
2,1367,Hollister's,493 Oakwood,Farmerville,GA,23146,6896.00,7500.0,268
3,1419,Everything Sports Shop,2939 Cardinal,Crystal,GA,23503,5396.36,5000.0,237
4,1462,Bargain House,4930 Main,Grove,GA,23321,4523.00,10000.0,268
5,1524,Jackson's,946 Second,Leonard,GA,23342,13817.00,15000.0,223
6,1619,Murray's Department Store,483 Cambridge,Sheldon,GA,23553,2217.00,10000.0,268
7,1687,Lawrence Sport and Appliance,393 Jefferson,Lafayette,GA,22543,3962.00,5000.0,237
8,1725,Dustin's All Seasons,171 Washington,Sheldon,GA,23553,359.00,7500.0,237
9,1842,Four Seasons Store,19 Front,Grove,GA,23321,8113.00,7500.0,223


In [ ]:
def run_query_dicts(query: str):
    try:
    # Connect to PostgreSQL
        conn = psycopg2.connect(**db_config)
        with conn.cursor(cursor_factory=RealDictCursor) as cur:
            cur.execute(query)
            if query.strip().lower().startswith('select'):
                return cur.fetchall()
            else:
                conn.commit()
                return f"{cur.rowcount} row(s) affected."
    except Exception as e:
        print("Error:", e)
run_query_dicts("SELECT * FROM product")